# Image Denoising

**There are 4 variations of Non-Local Means Denoising:**

- cv2.fastNlMeansDenoising() - works with a single grayscale images
- cv2.fastNlMeansDenoisingColored() - works with a color image.
- cv2.fastNlMeansDenoisingMulti() - works with image sequence captured in short period of time (grayscale images)
- cv2.fastNlMeansDenoisingColoredMulti() - same as above, but for color images.

In [1]:
import numpy as np
import cv2

image = cv2.imread('images/elephant.jpg')

# cv2.fastNlMeansDenoisingColored(input, None, h, hForColorComponents, templateWindowSize, searchWindowSize)
# None are - the filter strength 'h' (5-12 is a good range)
# Next is hForColorComponents, set as same value as h again
# templateWindowSize (odd numbers only) rec. 7
# searchWindowSize (odd numbers only) rec. 21

dst = cv2.fastNlMeansDenoisingColored(image, None, 11, 6, 7, 21)

cv2.imshow('Fast Means Denoising', dst)
cv2.waitKey(0)

cv2.destroyAllWindows()

# Image Inpainting

Inpainting is the process of reconstructing lost or deteriorated parts of images and videos. It is an advanced form of interpolation that can be used to replace lost or corrupted parts of the image data.

In [2]:
import cv2
import numpy as np

# Load our damaged photo
image = cv2.imread('images/abraham.jpg')
cv2.imshow('Original Damaged Photo', image)
cv2.waitKey(0)

# Load the photo where we've marked the damaged areas
marked_damages = cv2.imread('images/mask.jpg', 0)
cv2.imshow('Marked Damages', marked_damages)
cv2.waitKey(0)

# Let's make a mask out of our marked image be changing all colors 
# that are not white, to black
ret, thresh1 = cv2.threshold(marked_damages, 254, 255, cv2.THRESH_BINARY)
cv2.imshow('Threshold Binary', thresh1)
cv2.waitKey(0)

# Let's dilate (make thicker) our the marks w made
# since thresholding has narrowed it slightly
kernel = np.ones((7, 7), np.uint8)
mask = cv2.dilate(thresh1, kernel, iterations=1)
cv2.imshow('Dilated Mask', mask)
cv2.imwrite("images/abraham_mask.png", mask)

cv2.waitKey(0)
restored = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
# Radius of a circular neighborhood of each point inpainted that is considered by the algorithm,
# Smaller values look less blurred, while larger values look more pixelated or blurred.

cv2.imshow('Restored', restored)
cv2.waitKey(0)
cv2.destroyAllWindows()

# High Dynamic Range (HDR)

High-dynamic-range imaging (HDRI or HDR) is a technique used in imaging and photography to reproduce a greater dynamic range of luminosity than is possible with standard digital imaging or photographic techniques. While the human eye can adjust to a wide range of light conditions, most imaging devices use 8-bits per channel, so we are limited to only 256 levels. When we take photographs of a real world scene, bright regions may be overexposed, while the dark ones may be underexposed, so we can’t capture all details using a single exposure. HDR imaging works with images that use more than 8 bits per channel (usually 32-bit float values), allowing much wider dynamic range.

In [3]:
import cv2
import numpy as np
import os

def loadExposureSeq(path):
    images = []
    times = []
    with open(os.path.join(path, 'list.txt')) as f:
        content = f.readlines()
    for line in content:
        tokens = line.split()
        images.append(cv2.imread(os.path.join(path, tokens[0])))
        times.append(1 / float(tokens[1]))
    return images, np.asarray(times, dtype=np.float32)
    
# Load images and exposure times
images, times = loadExposureSeq('images/exposures/')

# Estimate camera response
calibrate = cv2.createCalibrateDebevec()
response = calibrate.process(images, times)

# Make HDR image
merge_debevec = cv2.createMergeDebevec()
hdr = merge_debevec.process(images, times, response)

# Tonemap HDR image
tonemap = cv2.createTonemap(2.2)    # Other ToneMap functions available
ldr = tonemap.process(hdr)

# Perform exposure fusion
merge_mertens = cv2.createMergeMertens()
fusion = merge_mertens.process(images)

# Convert to 8-bit
fusion_8bit = np.clip(fusion*255, 0, 255).astype('uint8')
ldr_8bit = np.clip(ldr*255, 0, 255).astype('uint8')
hdr_8bit = np.clip(hdr*255, 0, 255).astype('uint8')

# Displaying results
cv2.imshow('Fusion', fusion_8bit)
cv2.imshow('LDR', ldr_8bit)
cv2.imshow('HDR', hdr_8bit)

cv2.waitKey(0)
cv2.destroyAllWindows()